In [1]:
from PIL import Image
import numpy as np

In [2]:
def crop(ImgArr):
    i = 0
    while sum(sum(ImgArr[i])) == 0:
        ImgArr = np.delete(ImgArr, i, 0)
    i = -1
    while sum(sum(ImgArr[i])) == 0:
        ImgArr = np.delete(ImgArr, i, 0)
    i = 0
    while sum(sum(ImgArr[:,i])) == 0:
        ImgArr = np.delete(ImgArr, i, 1)
    i = -1
    while sum(sum(ImgArr[:,i])) == 0:
        ImgArr = np.delete(ImgArr, i, 1)
    return ImgArr

In [3]:
ALPHA = "ALPHA"
alpha_bit = 0xFE

def RGBA_to_bitmap(col):
    #8 BIT: BBBGGGRR
    if len(col) == 4:
        if col[3] == 0: return ALPHA
    col = np.array(col)/255.
    bitcol = int(round(col[0]*3) + round(col[1]*7)*4 + round(col[2]*7)*4*8)
    if bitcol == alpha_bit: bitcol += 1
    return bitcol

def bitmap_to_RGBA(bitcol):
    #8 BIT: BBBGGGRR
    if bitcol == ALPHA: return [0,0,0,0]
    R = bitcol%4
    G = bitcol//4 % 8
    B = bitcol//(4*8)
    return [int(x) for x in [R/3.*255, G/7.*255, B/7.*255, 255]]
    
def GetBitmap(ImgArr):
    shape = ImgArr.shape
    Bitmap = [[RGBA_to_bitmap(COL) for COL in row] for row in ImgArr]
    return {"X": shape[1], "Y": shape[0]}, Bitmap

def Bitmap_to_string(Bitmap):
    return ", ".join([str(item) for sublist in Bitmap for item in sublist])

def Bitmap_to_image(Bitmap):
    return Image.fromarray(np.array([[bitmap_to_RGBA(COL) for COL in row] for row in Bitmap], dtype="uint8"))

In [4]:
def Filename_to_bitmap(FILENAME, RESOLUTION_x, Orientation = "xy"):
    '''
    FILENAME must be of an image with an alpha channel
    RESOLUTION_x is an estimate of the x-resolution of the final bitmap
    
    returns:
        STRING, string of final bitmap. Just copy and paste it
        shape: shape of final bitmap (after cropping extra alpha rows and columns)
    '''
    
    Img = Image.open(FILENAME)
    RX = np.asarray(Img).shape[0]
    RY = np.asarray(Img).shape[1]
    if Orientation == "yx": t = RX; RX = RY; RY = t;
    scale = RESOLUTION_x/RX
    Img = Img.resize((round(RX*scale), round(RY*scale)), Image.NEAREST)
    ImgArr = np.asarray(Img)
    ImgArrCrop = crop(ImgArr)
    shape, Bitmap = GetBitmap(ImgArrCrop)
    STRING = Bitmap_to_string(Bitmap)
    preview = Bitmap_to_image(Bitmap)
    
    return STRING, shape, preview

In [5]:
FILENAME = r"C:\Users\nailu\Downloads\eheh.png"
FILENAME = r"C:\Users\nailu\Downloads\grass.png"
#FILENAME = r"C:\Users\nailu\Downloads\flag (3).jpg"
STRING, shape, preview = Filename_to_bitmap(FILENAME, 96, Orientation = "yx")

In [6]:
shape

{'X': 96, 'Y': 4}

In [327]:
preview

In [328]:
STRING

'ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, 48, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, 48, 44, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, 48, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, 48, 44, ALPHA, ALPHA, 44, 48, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, 48, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, 48, ALPHA, ALPHA, ALPHA, 80, 48, ALPHA, 48, 48, 53, 48, 48, ALPHA, 48, ALPHA, ALPHA, 53, ALPHA, 53, 53, ALPHA, ALPHA, ALPHA, ALPHA, 53, 48, 85, 48, 48, 80, ALPHA, ALPHA, 48, ALPHA, ALPHA, ALPHA, 80, 48, ALPHA, 48, 48, 53, 48, 48, ALPHA, 48, ALPHA, ALPHA, 53, ALPHA, 53, 53, ALPHA, ALPHA, ALPHA, ALPHA, 53, 48, 85, 48, 48, 80, ALPHA, ALPHA, 80, 48,

In [1]:
from PIL import GifImagePlugin

In [138]:
def replace_alpha(col, frame):
    frame = frame.copy();
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            if sum([1 for X,Y in zip(frame[i][j], col) if X != Y]) == 0:
                for k in range(4): frame[i][j][k] = 0
    return frame

In [333]:
FILENAME = r"C:\Users\nailu\Downloads\doggy.gif"
SPRITENAME = "DOGGY"
RESOLUTION_X = 30 

GIF = Image.open(FILENAME)
n_frames = GIF.n_frames
FRAMES = []
for i in range(n_frames):
    GIF.seek(i)
    img = GIF.copy()
    FRAMES.append(img)
    
scale = float(RESOLUTION_X)/np.asarray(FRAMES[0]).shape[1]
FRAMES_resized = [Img.resize((int(np.asarray(Img).shape[1] * scale), int(np.asarray(Img).shape[0] * scale)), Image.NEAREST) for Img in FRAMES]
FRAMES_RGBA = [np.asarray(Img.convert("RGBA")) for Img in FRAMES_resized]
FRAMES_RGBA = [replace_alpha([255, 255, 255, 255], frame) for frame in FRAMES_RGBA]
FRAMES_cropped = [crop(Img) for Img in FRAMES_RGBA]
Bitmap = [GetBitmap(Img) for Img in FRAMES_cropped]
STRING = [(shape, Bitmap_to_string(bitmap)) for shape, bitmap in Bitmap]

print(Bitmap[0][0])

CODE = '''
const unsigned char {0}_XY[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)*2) + "{" + ", ".join(["{0}, {1}".format(shape['X'],shape['Y']) for shape, _ in Bitmap]) + "};" + '''
''' + "\n".join(["const unsigned char {2}_{0}[{3}] PROGMEM = {1} ".format(i, "{" + string[1] + "};", SPRITENAME, Bitmap[i][0]['X']*Bitmap[i][0]['Y']) for i, string in enumerate(STRING)]) + '''

const long {0}_ARR[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)) + "{" + ", ".join(["{0}_{1}".format(SPRITENAME, i) for i in range(len(STRING))]) + "};"

{'X': 24, 'Y': 12}


In [335]:
with open("code.txt", "w") as f: f.write(CODE)

In [332]:
np.asarray(FRAMES_RGBA[1])

array([[[255, 255, 255, 255],
        [255, 255, 255, 255],
        [255, 255, 255, 255],
        ...,
        [255, 255, 255, 255],
        [255, 255, 255, 255],
        [255, 255, 255, 255]],

       [[255, 255, 255, 255],
        [255, 255, 255, 255],
        [255, 255, 255, 255],
        ...,
        [255, 255, 255, 255],
        [255, 255, 255, 255],
        [255, 255, 255, 255]],

       [[255, 255, 255, 255],
        [255, 255, 255, 255],
        [255, 255, 255, 255],
        ...,
        [255, 255, 255, 255],
        [255, 255, 255, 255],
        [255, 255, 255, 255]],

       ...,

       [[255, 255, 255, 255],
        [255, 255, 255, 255],
        [255, 255, 255, 255],
        ...,
        [255, 255, 255, 255],
        [255, 255, 255, 255],
        [255, 255, 255, 255]],

       [[255, 255, 255, 255],
        [255, 255, 255, 255],
        [255, 255, 255, 255],
        ...,
        [255, 255, 255, 255],
        [255, 255, 255, 255],
        [255, 255, 255, 255]],

       [[255

In [291]:
[b[0] for b in Bitmap]

[{'X': 17, 'Y': 8},
 {'X': 18, 'Y': 9},
 {'X': 19, 'Y': 8},
 {'X': 18, 'Y': 8},
 {'X': 17, 'Y': 7},
 {'X': 17, 'Y': 7},
 {'X': 18, 'Y': 7}]